<div style='background-image: url("./seissol2.jpg") ; padding: 0px ; background-size: cover ; border-radius: 5px ; height: 250px'>
    <div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.7) ; width: 50% ; height: 130px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.8) ; line-height: 100%">ASCETE</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.5)">3D Subduction fault simulation </div>
        </div>
    </div>
</div>

---

This notebook is based on a test case from ASCETE project for linking dynamic rupture and tsunami modeling in a simple, 3D subduction zone model.

More infomation about SeisSol can found at [@SeisSol GitHub](https://github.com/SeisSol)

##### Authors:
* Duo Li ([@dli](https://github.com/daisy20170101))
* Thomas Ulrich ([@tulrich](https://github.com/Thomas-Ulrich))
* Elizabeth Madden ([@elizabethmadden](betsymadden@gmail.com))
* Alice Gabriel([@agabriel](https://www.geophysik.uni-muenchende/Members/gabriel))

## Basic Equations

SeisSol is designed to solve for seismic wave propagation and dynamic earthquake rupture on an unstructured tetrahedral mesh.

SeisSol solves the linear elastic wave equation.
$$
\begin{align}
\frac{\partial}{\partial_t} \sigma_{xx} - (\lambda+2\mu)\frac{\partial}{\partial_x} u - \lambda \frac{\partial}{\partial_y} v -\lambda  \frac{\partial}{\partial_z} w &= S_{xx}\\\\
\frac{\partial}{\partial_t} \sigma_{yy} - \lambda \frac{\partial}{\partial_x} u - (\lambda+2\mu) \frac{\partial}{\partial_y} v - \lambda \frac{\partial}{\partial_z} w &= S_{yy}\\\\
\frac{\partial}{\partial_t} \sigma_{zz} - \lambda \frac{\partial}{\partial_x} u - \lambda \frac{\partial}{\partial_y} v - (\lambda+2\mu)\frac{\partial}{\partial_z} w &= S_{zz}\\\\
\frac{\partial}{\partial_t} \sigma_{xy} - \mu (\frac{\partial}{\partial_x}v +\frac{\partial}{\partial_y}u  ) &=  S_{xy} \\\\
\frac{\partial}{\partial_t} \sigma_{yz} - \mu (\frac{\partial}{\partial_z}v +\frac{\partial}{\partial_y}w  ) &=  S_{yz} \\\\
\frac{\partial}{\partial_t} \sigma_{xz} - \mu (\frac{\partial}{\partial_z}u +\frac{\partial}{\partial_x}w  ) &=  S_{xz} \\\\
\frac{\partial}{\partial_t} (\rho u) - \frac{\partial}{\partial_x} \sigma_{xx} - \frac{\partial}{\partial_y} \sigma_{xy} -  \frac{\partial}{\partial_z} \sigma_{xz} &= \rho S_u\\\\
\frac{\partial}{\partial_t} (\rho v) - \frac{\partial}{\partial_x} \sigma_{xy} - \frac{\partial}{\partial_y} \sigma_{yy} -  \frac{\partial}{\partial_z} \sigma_{yz} &= \rho S_v\\\\
\frac{\partial}{\partial_t} (\rho w) - \frac{\partial}{\partial_x} \sigma_{xz} - \frac{\partial}{\partial_y} \sigma_{yz} -  \frac{\partial}{\partial_z} \sigma_{zz} &= \rho S_w
\end{align}
$$



## SeisSol Initialization 

### Compilation specifications





### Parameter file

 The .par (an Ascii text file) constains the necessary parameters for SeisSol to run the earthquake model.

[An example](PAR_AscSimple_dip16_big_PfLith_mu4.txt)

#### Equation
Specify either elastic or plastic equation.

```c
MaterialFileName = 'material_ascSimple.yaml'
```
If compile with plastic=1 then 2 more parameters are needed:

Plasticity=0 and  Tv=0.05 


#### Boundaries
```bash
BC_fs = 1                               ! enable free surface boundaries

BC_dr = 1                               ! enable fault boundaries

BC_of = 1                               ! enable absorbing boundaries
```

#### DynamicRupture
Specity parameters related to dynamic rupture fault, e.g. friction paramters.

    FL = 30        ! Friction law  !0: none, 16:LSW, 103: RS with strong velocity weakening

    ModelFileName = 'initial_stress_ascSimple.dip16_PfLith_mu4.yaml'

    GPwise = 1                  ! elementwise =0 ; GPwise =1

    XRef = 1.0e9                ! Reference point

    YRef = 0.0 

    ZRef = 0.0 

    t_0 = 0.5

    RF_output_on = 1            ! RF on      

    magnitude_output_on =1

    energy_rate_output_on =1

    OutputPointType = 4         ! Type (0: no output, 1: take GP's 2: 4 points per surface triangle, 3: output at certain pickpoints, 4: fault surface output, 5=4+3)
    
    

#### Elementwise

Specify output variabls and parameters for each elelment. 

    printIntervalCriterion = 2       ! 1=iteration, 2=time

    printtimeinterval_sec = 1.0      ! Time interval at which output will be written

    OutputMask = 1 1 1 1 1 1 1 1 1 1 1  

    ! output 1/ yes, 0/ no     position: 1/ SRs and SRd: slip rates in strike and dip direction 
    ! 2/ T_s, T_d: transient shear stress in strike and dip direction, P_n: transient normal stress
    ! 3/ U_n: normal velocity (note that there is no fault opening in SeisSol)
    ! 4/ Mud: current friction, StV: state variable in case of RS friction
    ! 5/ Ts0,Td0,Pn0: total stress, including initial stress, 6/ Sls and Sld: slip in strike and dip direction
    ! 7/ Vr: rupture velocity, computed from the spatial derivatives of the rupture time
    ! 8/ ASl: absolute slip, 9/ PSR: peak slip rate, 10/ RT: rupture time, 11/DS: only with LSW, time at which ASl>D_c

    refinement_strategy = 2

    refinement = 0

#### Pickpoint

    printtimeinterval = 1       ! Index of printed info at timesteps -  was 10

    OutputMask = 1 1 1 0 1      ! output 1/ yes, 0/ no - position: 1/ slip rate 2/ stress 3/ normal velocity 4/ in case of rate and state output friction and state variable
                            ! 5/ background values
                            
    nOutpoints = 1

    PPFileName = 'hypo.dip18.dat'


#### SourceType

Setup point source if necessary.

#### SpongeLayer

#### MeshNml

Setup mesh type and file.


    MeshFile = 'AscSimple.big.100.04.025.r6.dip16'
    meshgenerator = 'PUML'       ! Name of meshgenerator (format)


#### Discretization

Spatical and time domain discretization.

    CFL = 0.5                       ! CFL number (<=1.0)
    FixTimeStep = 5                 ! Manualy chosen minimum tim
    ClusteredLTS = 2 		! Local time stepping



#### Output

Specify output

    OutputFile = 'output_PfLith_mu4/output_AscSimple_dip16_withCoh_PfLith_mu4'      
    !checkPointFile = '../OutputSumatraTestb/checkpoint'
    !checkPointInterval = 200    ! Set to 0 to disable checkpointing
    !checkPointBackend = 'hdf5'  ! Either ’hdf5’, ’mpio’ or ’none’
    !             |stress     |vel  |plast
    iOutputMask = 1 1 1 1 1 1 1 1 1 ! 0 0 0 0 0 0 0     ! Variables ouptut
    iOutputMaskMaterial = 1 1 1          ! Material output
    Format = 10                          ! Format (0=IDL, 1=TECPLOT, 2=IBM DX, 4=GiD, 6=hdf5, 10= no output))
    !Interval = 100000                    ! Index of printed info at timesteps
    TimeInterval = 5.0                   ! Index of printed info at time -  was 2
    !OutputRegionBounds = 100e3 1000e3 100e3 1600e3 -50e3 50e3
    printIntervalCriterion = 2           ! Criterion for index of printed info: 1=timesteps,2=time,3=timesteps+time
    SurfaceOutput = 1
    SurfaceOutputRefinement = 1
    SurfaceOutputInterval = 1.0
    pickdt = 0.1                         ! Pickpoint Sampling
    pickDtType = 1                       ! Pickpoint Type
    FaultOutputFlag = 1                 ! DR output (add this line only if DR is active)
    !nRecordPoints = 4783                   ! number of Record points which are read from file
    !RFileName = 'mesh161012b_orig_sameorder.dat'          ! Record Points in extra file


#### AbortCriteria

    EndTime = 300.0

#### Analysis

#### Debugging

### Easi

SeisSol employs easi files (.yaml) for assigning properties related to fault friction, the stress field and the surrounding material. It is advantageous to use yaml language for complicated parameter file.

These are explained below for this earthquake model. 

More examples can be found at [SeisSol:easi](https://seissol.readthedocs.io/en/latest/easi.html)




### Velocity structure

A materials .yaml contains the constant values assigned in this test case to density (rho), Lame's parameter (lambda) and the shear modulus (mu).

[The material yaml file](material_ascSimple.yaml)


### Fault friction

SeisSol provides a number of friction constitutive laws, e.g. linear slip weakening and rate-and-state. 

This test case utilizes linear slip weakening and this law (30) is assigned in the parameter file.
The frictional parameters (the static and dynamic friction coefficients, mu_s and mu_d, and the slip-weakening distance, d_c) are assigned in the stress .yaml file.
The on-fault cohesion is constant in Scenario B, but increases to 8 MPa above 15 km depth in Scenario A.



### Initial stress condition

SeisSol uses stress tensor ($\sigma_{xx},\sigma_{yy},\sigma_{zz},\sigma_{xy},\sigma_{yz},\sigma_{xz}$) or traction components (normal & shear stress) for initializing background stress condition. 

This test case has a simple, depth-dependent stress field specified in AscSimple_initial_stress.yaml.
The pore fluid pressure also is specified in the stress .yaml and is near-lithostatic.


<img src="AscSimple_PfTnTsMuDcCohStr_New_cmu.jpg"
     alt="Inputs for the earthquake model in the test case."
     style="float: left; margin-right: 10px;height:300px;" />
     




### Mesh generation


#### GMSH - a free mesh generator

The .geo is the GMSH file for the earthquake model mesh and can be editted directly to change the structural model or the mesh resolution.
The .geo file is then optimized and converted to .neu format.
PUMGen is used to export the mesh into an efficient xdmf format that is used by SeisSol.

More GMSH information can be found at [SeisSol: Gmsh](https://seissol.readthedocs.io/en/latest/gmsh.html)
More PUMGen information can be found at [SeisSol: Meshing with PUMGen](https://seissol.readthedocs.io/en/latest/meshing-with-pumgen.html)


<img src="AscSimple_geo.png"
     alt="Image of mesh structure defined in the .geo file"
     style="float: left; margin-right: 10px;height:300px;" />
     




#### Simmodeler - a commercial mesh generator


A detailed instruction can be found at [SeisSol: GOCAD model](https://seissol.readthedocs.io/en/latest/meshing-with-simmodeler.html)

### Launch SeisSol in MPI Cluster

Prerequisite module needs:

- Python (>= 3.6)
- Numpy (>= 1.12.0)
- SCons (>= 3.0, for instructions see below)
- hdf5 (>= 1.8, for instructions see below)
- netcdf (C-Release) (>= 4.4, for instructions see below)
- Intel compiler (>= 17.0, icc, icpc, ifort) or GCC (>= 5.0, gcc, g++, gfortran)
- Some MPI implementation (e.g. OpenMPI)
- ParMETIS for partitioning
- libxsmm (libxsmm_gemm_generator) for small matrix multiplications
- PSpaMM (pspamm.py) for small sparse matrix multiplications (required only on Knights Landing or Skylake)
- CMake (for compiling submodules ImpalaJIT and yaml-cpp)


## Launch SeisSol in ICTP Argo

```bash
export PATH=$PATH:$HOME/bin::/home/dli/software/bin
export LIBRARY_PATH=/home/dli/software/lib:$LIBRARY_PATH
export LD_LIBRARY_PATH=/home/dli/software/lib:$LD_LIBRARY_PATH

#module for SeisSol
module load intel-mkl/2017
module load intel/2017
module load openmpi/1.10.2/intel/2017
module load python/3.6.4
module load cmake/3.9.2

#compiler for SeisSol in openmpi/1.10.2/intel/2017
export CC=mpicc
export FC=mpif90
export CXX=mpicxx
```

## Run Seissol


    echo $dir_to_SeisSol/Maple/ > DGPATH
    mpirun -n np ./seissol_excutable parameters.par



## Result Visualization

SeisSol provides results in vtk format that is used in [ParaView](https://www.paraview.org/) directly. 



A collection of useful scripts can be found in [SeisSol](https://github.com/SeisSol/Visualization). 